Now that we have learned several methods and explored them with simple examples, we will try them out on a real example: the MNIST digits.

We can load this data using the following **dslabs** package:


In [1]:
library(dslabs)
if (!exists("mnist")) mnist <- read_mnist()

The dataset includes two components, a training set and a test set:

In [2]:
names(mnist)

Each of these components includes a matrix with features in the columns:

In [3]:
dim(mnist$train$images)

and vector with the classes as integers:

In [4]:
class(mnist$train$labels)
table(mnist$train$labels)

Because we want this example to run on a small laptop and in less than one hour, we will consider a subset of the dataset. We will sample 1,000 random rows from the training set and 500 random rows from the test set:

In [6]:
set.seed(1990)
index <- sample(nrow(mnist$train$images), 1000)
x <- mnist$train$images[index,]
y <- factor(mnist$train$labels[index])
index <- sample(nrow(mnist$test$images), 500)
x_test <- mnist$test$images[index,]
y_test <- factor(mnist$test$labels[index])

## The caret package

We have already learned about several machine learning algorithms. Many of these algorithms are implemented in R. However, they are distributed via different packages, developed by different authors, and often use different syntax. The __caret__ package tries to consolidate these differences and provide consistency. It currently includes over 200 different methods which are summarized in the __caret__ package manual^[https://topepo.github.io/caret/available-models.html]. Keep in mind that __caret__ does not include the packages needed to run each possible algorithm. To apply a machine learning method through __caret__ you still need to install the library that implement the method. The required packages for each method are described in the package manual. 

The __caret__ package also provides a function that performs cross validation for us. Here we provide some examples showing how we use this helpful package. We will first use the 2 or 7 example to illustrate and, in later sections, we use the package to run algorithms on the larger MNIST dataset.

### The `train` function 

The R functions that fit machine algorithms are all slightly different. Functions such as `lm`, `glm`, `qda`, `lda`, `knn3`, `rpart` and `randomForrest` use different syntax, have different argument names and produce objects of different types.

The __caret__ `train` function lets us train different algorithms using similar syntax. So, for example, we can type:

In [7]:
library(caret)
train_glm <- train(y ~ ., method = "glm", data = mnist_27$train)
train_qda <- train(y ~ ., method = "qda", data = mnist_27$train)
train_knn <- train(y ~ ., method = "knn", data = mnist_27$train)

As we explain in more detail in @sec-caret-cv, the `train` function selects parameters for you using a resampling method, with boostrap as the default.

### The `predict` function

The `predict` function is very useful for machine learning applications. This function takes an object from a fitting function and a data frame with features $\mathbf{x}$ for which to predict, and returns predictions for these features. 

Here is an example with logistic regression:

In [8]:
fit <- glm(y ~ ., data = mnist_27$train, family = "binomial")
p_hat <- predict(fit, mnist_27$test)

In this case, the function is simply computing: 

$$
\hat{p}(\mathbf{x}) = g^{-1}\left(\hat{\beta}_0 + \hat{\beta}_1 x_1 + \hat{\beta}_2 x_2 \right) \text{ with } g(p) = \log\frac{p}{1-p} \implies g^{-1}(\mu) = \frac{1}{1-e^{-\mu}}
$$

for the `x_1` and `x_2` in the test set `mnist_27$test`. With these estimates in place, we can make our predictions and compute our accuracy:

In [9]:
y_hat <- factor(ifelse(p_hat > 0.5, 7, 2))

However, note that `predict` does not always return objects of the same types; it depends on what type of object it is applied to. To learn about the specifics, you need to look at the help file specific for the type of fit object that is being used. The `predict` is actually a special type of function in R (called a _generic function_) that calls other functions depending on what kind of object it receives. So if `predict` receives an object coming out of the `lm` function, it will call `predict.glm`. If it receives an object coming out of `glm`, it calls `predict.qda`. If the fit is from `knn3`, it calls `predict.knn3`, and so on. These functions are similar but not exactly. You can learn more about the differences by reading the help files:

In [10]:
?predict.glm
?predict.qda
?predict.knn3

There are many other versions of `predict` and many machine learning algorithms have a `predict` function.

As with `train`, the **caret** packages unifies the use of `predict` with the function `predict.train`. This function takes the output of `train` and produces prediction of categories or estimates of $p(\mathbf{x})$.

The code looks the same for all methods:

In [11]:
y_hat_glm <- predict(train_glm, mnist_27$test, type = "raw")
y_hat_qda <- predict(train_qda, mnist_27$test, type = "raw")
y_hat_knn <- predict(train_knn, mnist_27$test, type = "raw")

This permits us to quickly compare the algorithms. For example, we can compare the accuracy like this:

In [12]:
fits <- list(glm = y_hat_glm, qda = y_hat_qda, knn = y_hat_knn)
sapply(fits, function(fit) confusionMatrix(fit, mnist_27$test$y)$overall[["Accuracy"]])

### Cross validation

When an algorithm includes a tuning parameter, `train` automatically uses cross validation to decide among a few default values. To find out what parameter or parameters are optimized, you can read the manual ^[http://topepo.github.io/caret/available-models.html] or study the output of: 

In [13]:
getModelInfo("knn")

We can also use a quick lookup like this:

In [0]:
modelLookup("knn")

If we run it with default values: 

In [14]:
train_knn <- train(y ~ ., method = "knn", data = mnist_27$train)

you can quickly see the results of the cross validation using the `ggplot` function. The argument `highlight` highlights the max:

In [15]:
ggplot(train_knn, highlight = TRUE)

By default, the cross validation is performed by taking 25 bootstrap samples comprised of 25% of the observations. For the `kNN` method, the default is to try $k=5,7,9$. We change this using the `tuneGrid` argument. The grid of values must be supplied by a data frame with the parameter names as specified in the `modelLookup` output. 

Here we present an example where we try out 30 values between 9 and 67. To do this with __caret__, we need to define a column named `k`, so we use this:  `data.frame(k = seq(9, 67, 2))`. Note that when running this code, we are fitting 30 versions of kNN to 25 bootstrapped samples. Since we are fitting $30 \times 25 = 750$ kNN models, running this code will take several seconds. We set the seed because cross validation is a random procedure and we want to make sure the result here is reproducible.

In [16]:
set.seed(2008)
train_knn <- train(y ~ ., method = "knn", 
                   data = mnist_27$train,
                   tuneGrid = data.frame(k = seq(9, 71, 2)))
ggplot(train_knn, highlight = TRUE)

To access the parameter that maximized the accuracy, you can use this:

In [17]:
train_knn$bestTune

and the best performing model like this:

In [18]:
train_knn$finalModel

The function `predict` will use this best performing model. Here is the accuracy of the best model when applied to the test set, which we have not yet used because the cross validation was done on the training set:

In [19]:
confusionMatrix(predict(train_knn, mnist_27$test, type = "raw"),
                mnist_27$test$y)$overall["Accuracy"]

If we want to change how we perform cross validation, we can use the `trainControl` function. We can make the code above go a bit faster by using, for example, 10-fold cross validation. This means we have 10 samples using 10% of the observations each. We accomplish this using the following code:

In [20]:
control <- trainControl(method = "cv", number = 10, p = .9)
train_knn_cv <- train(y ~ ., method = "knn", 
                   data = mnist_27$train,
                   tuneGrid = data.frame(k = seq(9, 71, 2)),
                   trControl = control)
ggplot(train_knn_cv, highlight = TRUE)

We observe that the accuracy estimates are more variable, which is expected since we changed the number of samples used to estimate accuracy.

Note that `results` component of the `train` output includes several summary statistics related to the variability of the cross validation estimates:

In [21]:
names(train_knn$results)

You can learn many more details about the **caret** package, from the manual ^[https://topepo.github.io/caret/available-models.html].

## Preprocessing

We often transform predictors before running the machine algorithm. We also remove predictors that are clearly not useful. We call these steps *preprocessing*.

Examples of preprocessing include standardizing the predictors, taking the log transform of some predictors, removing predictors that are highly correlated with others, and removing predictors with very few non-unique values or close to zero variation. 

For example, we can run the `nearZero` function from the **caret** package to see that several features do not vary much from observation to observation. We can see that there is a large number of features with close to 0 variability:

In [22]:
library(matrixStats)
sds <- colSds(x)
hist(sds, breaks = 256)

This is expected because there are parts of the image that rarely contain writing (dark pixels).

The **caret** packages includes a function that recommends features to be removed due to *near zero variance*:

In [23]:
nzv <- nearZeroVar(x)

We can see the columns recommended for removal are the near the edges:

In [24]:
image(matrix(1:784 %in% nzv, 28, 28))

So we end up keeping this number of columns:

In [25]:
col_index <- setdiff(1:ncol(x), nzv)
length(col_index)

Now we are ready to fit some models. Before we start, we need to add column names to the feature matrices as these are required by **caret**:

In [26]:
colnames(x) <- 1:ncol(mnist$train$images)
colnames(x_test) <- colnames(x)

## k-nearest neighbors

\Before starting this section, note that the first two calls to the `train` function in the code below can take several hours to run. This is a common challenge when training machine learning algorithms since we have to run the algorithm for each cross validation split and each set of tuning parameters being considered. In the next section, we will provide some suggestions on how to predict the duration of the process and ways to reduce. 

The first step is to optimize for $k$.

In [27]:
train_knn <- train(x[ ,col_index], y, 
                   method = "knn", 
                   tuneGrid = data.frame(k = seq(3, 13, 2)))

Once we optimize our algorithm, we can fit it to the entire dataset:

In [29]:
fit_knn <- knn3(x[, col_index], y,  k = train_knn$bestTune)

We achieve a high accuracy:

In [30]:
y_hat_knn <- predict(fit_knn, x_test[, col_index], type = "class")
confusionMatrix(y_hat_knn, factor(y_test))$overall["Accuracy"]

## Testing and improving computation time

The default method for estimating accuracy used by the `train` function is to test prediction on 25 bootstrap samples. In some cases, 25 bootraps samples might not be enough to obtain a precise estimate of MSE. We can improve precision by increasing the number of bootstrap samples. However, this will increase computation time. For example, if we are considering several values, say 10, of the tuning parameters, we will fit the algorithm 250 times. 

This can be a challenge for machine learning algorithms such as Random Forests, which are particular computationally intensive.
We can use the `system.time` function to estimate how long it takes to run the algorithm once. 

In [31]:
library(randomForest)
system.time({fit_rf <- randomForest(x[, col_index], y,  mtry = 9)})

and use this to estimate the total time for the 250 iterations. In this case, a call like the following would take 250 times longer. 

**Do not run the following command unless you are ready to wait several minutes.**

In [0]:
system.time({
  train_rf <- train(x[, col_index], y, 
    method = "rf", 
    tuneGrid = data.frame(mtry = seq(4, 24, 5)))
})

One way to reduce run time is to use k-fold cross validation with a smaller number of test sets. A popular choice is leaving out 10 test sets with 10% of the data:

In [32]:
control <- trainControl(method = "cv", number = 10, p = .9)

and re-running the `train` function with this choice specified via the `trControl` argument. For example, 

In [0]:
system.time({
  train_rf_cv <- train(x[, col_index], y, 
                       method = "rf", 
                       tuneGrid = data.frame(mtry = seq(4, 24, 5)),
                       trControl = control)
})
print(train_rf_cv$bestTune$mtry)

However, remember that this gives a different estimate of MSE and thus could result in a different choice of best model.

For random forest, we can also speed up the training step by running less trees per fit. After running the algorithm once, we can use the plot function to see how the error rate changes as the number of trees grows. 

Here we can see that error rate stabilizes after about 200 trees:

In [33]:
plot(fit_rf)

We can use this finding to speed up the cross validation procedure. Specifically, because the default is 500, by adding the argument `ntree = 150` to the call to `train` above, the procedure will finish 3.3 times faster.

In [34]:
system.time({
  train_rf_cv_150 <- train(x[, col_index], y, 
                       method = "rf", 
                       tuneGrid = data.frame(mtry = seq(4, 24, 5)),
                       trControl = control,
                       ntree = 150)
})
print(train_rf_cv_150$bestTune$mtry)

## Variable importance

The following function computes the importance of each feature:

In [35]:
imp <- importance(fit_rf)

We can see which features are being used most by plotting an image:

In [36]:
mat <- rep(0, ncol(x))
mat[col_index] <- imp
image(matrix(mat, 28, 28))

## Diagnostics

An important part of data analysis is visualizing results to determine why we are failing. How we do this depends on the application. Below we show the images of digits for which we made an incorrect prediction. Here are some errors for the random forest:

In [37]:
p_max <- predict(fit_knn, x_test[,col_index], type = "prob") 
p_max <- p_max / rowSums(p_max)
p_max <- apply(p_max, 1, max)

ind  <- which(y_hat_knn != y_test)
ind <- ind[order(p_max[ind], decreasing = TRUE)]

par(mfrow=c(1,4))
for (i in ind[1:4]) {
  image(matrix(x_test[i,], 28, 28)[, 28:1], 
        main = paste0("Pr(",y_hat_knn[i],")=",round(p_max[i], 2), " but is a ",y_test[i]),
        xaxt = "n", yaxt = "n")
}

By examining errors like this, we often find specific weaknesses to algorithms or parameter choices and can try to correct them.

## Ensembles

The idea of an ensemble is similar to the idea of combining data from different pollsters to obtain a better estimate of the true support for each candidate.

In machine learning, one can usually greatly improve the final results by combining the results of different algorithms.

Here is a simple example where we compute new class probabilities by taking the average of random forest and kNN. We can see that the accuracy improves:

In [38]:
p_rf <- predict(fit_rf, x_test[,col_index], type = "prob")  
p_rf <- p_rf / rowSums(p_rf)
p_knn  <- predict(fit_knn,  x_test[,col_index],  type="prob")
p <- (p_rf + p_knn)/2
y_pred <- factor(apply(p, 1, which.max) - 1)
confusionMatrix(y_pred, y_test)$overall["Accuracy"]

We have just built an ensemble with just two algorithms. By combing more similarly performing, but uncorrelated, algorithms we can improve accuracy further.